## Reciprocity Analysis: Data Cleaning
Viki Zygouras - 5/19/2020 

In [62]:
import pandas as pd
import glob
import os 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import numpy as np
import statsmodels.api as sm

warnings.filterwarnings('ignore')
pd.set_option("display.max_rows", None, "display.max_columns", None)

Read in files: 

In [63]:
path = r'reciprocityLargeFamilyData/'                    
all_files = glob.glob(os.path.join(path, "*.xlsx"))    
df_from_each_file = [pd.read_excel(f) for f in all_files]
totalData = pd.DataFrame()

In [64]:
#clean/drop any unnecessary columns 
reciprocityData = totalData.append(df_from_each_file, ignore_index= False)

reciprocityData = reciprocityData.drop(columns = ['anastasia code', 'emotion','positive/negative', 'Unnamed: 3',
                                                 'Anastasia Code','Postive or Negative?', 'Anastasia Coding',
                                                 'Anastasia Codes', 'Emotion', 'Positive / Negative'])
reciprocityData = reciprocityData.reset_index(drop = True)
reciprocityData = reciprocityData.drop_duplicates()
reciprocityData = reciprocityData.dropna(subset=['Title'])

## Data Cleaning

Parse out interaction labels:

In [65]:
#Split the participant code from label
reciprocityData["Participant"] = [val[0] for val in reciprocityData["Title"].str.partition("-", False)]

In [66]:
#Split agent name from label 
reciprocityData["Agent"] = [val[2].partition("-")[0].lower() for val in reciprocityData["Title"].str.partition("-", False)]

In [67]:
#Split either Active or Observing from the label 
reciprocityData['Activity'] = [val[2] for val in reciprocityData["Title"].str.split("-", False)]

In [68]:
#Boolean 1 if verbal 0 if nonverbal 
reciprocityData['Verbal'] = [val[3] for val in reciprocityData["Title"].str.split("-")]
reciprocityData['Verbal'] = [0 if elt == '26' else 1 for elt in reciprocityData['Verbal']]

In [69]:
#Split behavior number from label
reciprocityData['Behavior'] = [val[4] for val in reciprocityData["Title"].str.split("-")]
reciprocityData = reciprocityData.astype({'Behavior': 'int64'})

Will use 1 to indicate positive interaction and 0 to indicate negative. 999 means the value within our coding scheme should not be assigned to the following categories.

In [70]:
#Parse out positive and not positive - will be 1 if positive and 0 if not positive 
reciprocityData['Positive'] = ['E' if val[-2] != 'P' and val[-2] != 'NP' else val[-2] for val in reciprocityData["Title"].str.split("-")]
reciprocityData['Positive'] = reciprocityData['Positive'].map({'P': 1, 'NP': 0, 'E': 999})

In [71]:
#Parse out if the agent and participant's sentiment matches or not 
reciprocityData['InSync'] = ['E' if val[-1] != 'M' and val[-1] != 'S' else val[-1] for val in reciprocityData["Title"].str.split("-")]
reciprocityData['InSync'] = reciprocityData['InSync'].map({'S': 1, 'M': 0, 'E': 999})

Correct previously defined verbal/nonverbal to behaviors account for human error:  

In [72]:
#assign 0 to physical behaviors except for 23 
for i in range(7,29):
    if i != 23:
        reciprocityData['Verbal'] = np.where((reciprocityData.Behavior == i), 0, reciprocityData.Verbal)

In [73]:
#assign 1 to verbal behaviors
verbalBehaviorNum = [1,2,3,4,5,6,29,30,31,32,33,34,35,36,37]
for i in verbalBehaviorNum:
    reciprocityData['Verbal'] = np.where((reciprocityData.Behavior == i), 1, reciprocityData.Verbal)

In [74]:
#check that the above code corrected for human errors in behavior 11
reciprocityData[reciprocityData['Behavior'] == 11]

,Start,End,Title,Participant,Agent,Activity,Verbal,Behavior,Positive,InSync
54,15:15:51:22,15:15:54:09,P27-Computer-Active-26-11-NP-M,P27,computer,Active,0,11,0,0
55,15:15:52:02,15:15:54:14,P28-Computer-Obs-26-11-NP-M,P28,computer,Obs,0,11,0,0
84,15:18:44:15,15:18:53:14,P28-Jibo-Active-26-11-NP-S,P28,jibo,Active,0,11,0,1
607,13:08:14:22,13:08:17:02,P1-Jibo-Active-26-11-NP-S,P1,jibo,Active,0,11,0,1
647,13:19:19:03,13:19:22:14,P2-Computer-Obs-26-11-NP-M,P2,computer,Obs,0,11,0,0
675,09:47:23:092,09:47:28:076,P4-Jibo-Obs-26-11-NP-M,P4,jibo,Obs,0,11,0,0
704,09:54:17:104,09:54:22:116,P4-Computer-Obs-26-11-NP-M,P4,computer,Obs,0,11,0,0
1119,18:15:19:00,18:15:22:01,P16-Computer-Obs-26-11-NP-S,P16,computer,Obs,0,11,0,1
1157,18:20:51:06,18:20:53:26,P16-Computer-Obs-26-11-NP-S,P16,computer,Obs,0,11,0,1
1158,18:20:51:06,18:20:53:26,P16-Computer-Obs-26-11-25-NP-S,P16,computer,Obs,0,11,0,1


Update tags above for behaviors that shouldn't be considered for P/NP and M/S:

In [75]:
notincluded = [7,8,9,12,27,29]
for i in notincluded:
    reciprocityData['Positive'] = np.where((reciprocityData.Behavior == i), 999, reciprocityData.Positive)
    reciprocityData['InSync'] = np.where((reciprocityData.Behavior == i), 999, reciprocityData.InSync)
    reciprocityData['Verbal'] = np.where((reciprocityData.Behavior == i), 999, reciprocityData.Verbal)

Adding duration of a given interaction:

In [76]:
reciprocityData['EndTime'] =  pd.to_datetime(reciprocityData['End'], format = '%H:%M:%S:%f')
reciprocityData['StartTime'] =  pd.to_datetime(reciprocityData['Start'], format = '%H:%M:%S:%f')
reciprocityData['Duration'] = reciprocityData['EndTime'] - reciprocityData['StartTime']
reciprocityData = reciprocityData.drop(['StartTime', 'EndTime', 'Title'], axis = 1)

Average Duration (Seconds) of an Interaction: 

In [77]:
#drop subclips that weren't exported correctly from KYNO for average duration calculation 
incorrectlyExported = reciprocityData[reciprocityData['Duration'] > '00:01:00.000000']

In [78]:
#clips under the 1 minute threshold removes outlier duration clips 
actualDurations = reciprocityData[reciprocityData['Duration'] < '00:01:00.000000']
actualDurations['Duration'].mean()

Timedelta('0 days 00:00:07.327982')

Add info from "Participants.csv" in Study 3 Data Analysis and "behaviors.xls" in Behavioral Analysis: 

In [79]:
participants = pd.read_excel(r"Participants.xlsx")
behaviors = pd.read_excel(r"behaviors.xlsx")

In [80]:
pData = pd.DataFrame()
pData = pData.append([participants['Participant'],participants['Type'], participants['Family'] , participants['Age']]).T
pData = pData.iloc[:38]

In [81]:
temp_combined = pd.merge(pData, reciprocityData, on=['Participant'])
combined = pd.merge(temp_combined, behaviors, on = ['Behavior'])

Convert categorical columns to numeric with one-hot encoding:

In [82]:
combined['isJibo'] = [1 if value == 'jibo' else 0 for value in combined['Agent']]
combined['isComputer'] = [1 if value == 'computer' else 0 for value in combined['Agent']]
combined['isAlexa'] = [1 if value == 'alexa' else 0 for value in combined['Agent']]

combined['isChild'] = [1 if value == 'C' else 0 for value in combined['Type']]
combined['isAdult'] = [1 if value == 'A' else 0 for value in combined['Type']]

combined['isActive'] = [1 if value == 'Active' else 0 for value in combined['Activity']]
combined['isObserving'] = [1 if value == 'Obs' else 0 for value in combined['Activity']]

View raw data:

In [83]:
combined.tail(20)

,Participant,Type,Family,Age,Start,End,Agent,Activity,Verbal,Behavior,Positive,InSync,Duration,Description,Acknowledgement Verbal,Body,Head,Hand,Eye,Face,Positive Verbal,Negative Verbal,isJibo,isComputer,isAlexa,isChild,isAdult,isActive,isObserving
1691,P13,A,F04,51,15:55:48:27,15:55:50:08,computer,Active,1,31,0,1,00:00:01.810000,Thank You,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0
1692,P15,C,F05,12,18:29:16:07,18:29:18:18,jibo,Active,1,31,0,1,00:00:02.110000,Thank You,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0
1693,P15,C,F05,12,18:36:37:06,18:36:43:29,jibo,Active,1,31,1,1,00:00:06.230000,Thank You,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0
1694,P16,A,F05,46,18:14:48:27,18:14:56:06,jibo,Obs,1,31,1,0,00:00:07.790000,Thank You,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1
1695,P16,A,F05,46,18:34:46:08,18:34:57:20,jibo,Active,1,31,1,0,00:00:11.120000,Thank You,0,0,0,0,0,0,1,0,1,0,0,0,1,1,0
1696,P19,A,F06,49,12:58:32:06,12:58:38:10,alexa,Active,1,31,0,1,00:00:06.040000,Thank You,0,0,0,0,0,0,1,0,0,0,1,0,1,1,0
1697,P22,A,F07,56,13:16:24:036,13:16:31:040,alexa,Obs,1,31,0,1,00:00:07.004000,Thank You,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1
1698,P22,A,F07,56,13:17:12:020,13:17:12:104,alexa,Active,1,31,1,1,00:00:00.084000,Thank You,0,0,0,0,0,0,1,0,0,0,1,0,1,1,0
1699,P24,C,F08,6,16:12:28:11,16:12:31:20,jibo,Active,1,31,1,1,00:00:03.090000,Thank You,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0
1700,P24,C,F08,6,16:13:06:19,16:13:08:09,jibo,Active,1,31,1,1,00:00:01.900000,Thank You,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0


Export dataframe as CSV to make a copy: 

In [84]:
combined.to_csv("ReciprocityAnalysis.csv", index = False)

### General Metrics:

In [85]:
print("Number of clips coded: ", len(combined))

Number of clips coded:  1711


In [86]:
print("Number of Families: ", len(set(combined['Family'])))

Number of Families:  12


In [87]:
print("Families Included: ", set(combined['Family']))

Families Included:  {'F13', 'F06', 'F05', 'F17', 'F08', 'F01', 'F04', 'F02', 'F11', 'F10', 'F03', 'F07'}


In [88]:
print("Number of Participants: ", len(set(combined['Participant'])))

Number of Participants:  33


In [89]:
print("Average # of subclips per family: ", len(combined)/12)

Average # of subclips per family:  142.58333333333334


In [90]:
set(combined['Description'])

{'Angry Verbal',
 'Answer Question',
 'Compliment',
 'Defend',
 'Downward Frown',
 'Exclamation',
 'Follow-Up Question',
 'Generic Dance',
 'Glance at Others',
 'Hand Gesture',
 'Imitation',
 'Imitation Dance',
 'Inquisitive Eyebrow',
 'Insult',
 'Interrupt ',
 'Judgemental Eyebrow',
 'Judgemental Verbal',
 'Laugh',
 'Look Away',
 'Luring',
 'Mhm',
 'Mouth Open',
 'Move Away',
 'Move Toward',
 'Nod ',
 'Point',
 'Relevancy',
 'Shake Head ',
 'Short Response',
 'Shrug',
 'Smile',
 'Switch Eye Contact',
 'Thank You',
 'Tilt Head',
 'Touch ',
 'Waving',
 'Whisper'}

View negative interactions from Family 2

In [91]:
rel = combined[combined['Positive'] == 0]
rel[rel['Family'] == 'F02'].tail(10)

,Participant,Type,Family,Age,Start,End,Agent,Activity,Verbal,Behavior,Positive,InSync,Duration,Description,Acknowledgement Verbal,Body,Head,Hand,Eye,Face,Positive Verbal,Negative Verbal,isJibo,isComputer,isAlexa,isChild,isAdult,isActive,isObserving
847,P4,C,F02,7,10:21:37:14,10:21:41:00,alexa,Obs,1,6,0,1,00:00:03.860000,Relevancy,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1
945,P5,A,F02,44,10:21:57:11,10:22:00:23,jibo,Active,0,21,0,0,00:00:03.120000,Switch Eye Contact,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0
1120,P5,A,F02,44,09:47:22:048,09:47:27:080,jibo,Active,0,20,0,0,00:00:05.032000,Hand Gesture,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0
1121,P5,A,F02,44,09:49:21:020,09:49:24:100,jibo,Active,0,20,0,0,00:00:03.080000,Hand Gesture,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0
1122,P5,A,F02,44,09:50:22:080,09:50:32:112,jibo,Obs,0,20,0,0,00:00:10.032000,Hand Gesture,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1
1143,P4,C,F02,7,10:20:33:16,10:20:38:16,computer,Active,0,10,0,1,00:00:05,Point,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0
1389,P4,C,F02,7,10:20:33:16,10:20:38:16,computer,Active,1,5,0,1,00:00:05,Exclamation,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0
1409,P4,C,F02,7,10:21:24:27,10:21:31:26,computer,Obs,1,2,0,1,00:00:06.990000,Mhm,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1
1421,P5,A,F02,44,09:47:22:048,09:47:27:080,jibo,Active,1,35,0,0,00:00:05.032000,Judgemental Verbal,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0
1515,P5,A,F02,44,10:21:57:11,10:22:00:23,jibo,Active,0,26,0,0,00:00:03.120000,Inquisitive Eyebrow,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0


## Fisher Test Data: 

<span style="color:red">NOTE: There are no libraries in python that currently support Fisher tests for tables larger than 2x2 - need to use R instead. </span>

Build and Export Contingency Tables for fisher testing in R:

In [92]:
behaviors = list(set(combined['Description']))

In [93]:
#will make the count table for number of participants that exhibit a behavior with an agent (behavior x agent) 
agents = ['jibo', 'alexa', 'computer']
def makeContingencyTable(df):
    
    table = []
    for agent in agents: 
        agentCounts = pd.DataFrame(columns = [agent])
        labels = pd.DataFrame(columns = ['Description'])
        for i in range(len(behaviors)): 
            agentDf = df[df['Agent'] == agent]
            behaviorDf = agentDf[agentDf['Description'] == behaviors[i]]  
            numParticipants = len(set(behaviorDf['Participant']))
            agentCounts.loc[i] = [numParticipants]
            labels.loc[i] = [behaviors[i]]
        table.append(agentCounts)
        
    table.append(labels)
        
    combo = pd.concat(table, axis = 1)
    combo = combo[['Description', 'jibo', 'alexa', 'computer']]
    return combo


In [94]:
#use this code to make contigency table based on behavior frequency and not particiants (behavior x agent) 
# def makeContingencyTable(df):
#     data = df[['Description', 'Agent']]
#     table = sm.stats.Table.from_data(data)
#     contingencyTable = table.table_orig
#     return contingencyTable

#Run these to get the complement for behavior counts 
#calculate the total number of interactions per agent 
# alexaTotal = len(combined[combined['Agent'] == 'alexa'])
# jiboTotal = len(combined[combined['Agent'] == 'jibo'])
# computerTotal = len(combined[combined['Agent'] == 'computer'])
# jiboComplement = general['jibo'].apply(lambda x: jiboTotal - x)
# alexaComplement = general['alexa'].apply(lambda x: alexaTotal - x)
# computerComplement = general['computer'].apply(lambda x: computerTotal - x)

Create tables for each category:

In [95]:
#all behaviors no splits
general = makeContingencyTable(combined)

In [96]:
jiboComplement = general['jibo'].apply(lambda x: 33 - x)
alexaComplement = general['alexa'].apply(lambda x: 33 - x)
computerComplement = general['computer'].apply(lambda x: 33 - x)

In [97]:
complement = pd.concat([general['Description'], jiboComplement, alexaComplement, computerComplement], axis = 1)

Using 1 and 0 to filter on category columns ensures that ineligible behaviors (999) aren't included 

In [98]:
notPositive = combined[combined['Positive'] == 0]
notPositive = makeContingencyTable(notPositive)

In [99]:
positive = combined[combined['Positive'] == 1]
positive = makeContingencyTable(positive)

In [100]:
verbal = combined[combined['Verbal'] == 1]
verbal = makeContingencyTable(verbal)

In [101]:
nonverbal = combined[combined['Verbal'] == 0]
nonverbal = makeContingencyTable(nonverbal)

In [102]:
sync = combined[combined['InSync'] == 1]
sync = makeContingencyTable(sync)

In [103]:
mismatch = combined[combined['InSync'] == 0]
mismatch = makeContingencyTable(mismatch)

In [104]:
tableNames = ['General','Complement', 'Positive', 'Not Positive', 'InSync', 'Mismatch', 'Verbal', 'NonVerbal']
tableList = [general, complement, positive, notPositive, sync, mismatch, verbal, nonverbal]

In [105]:
positive

,Description,jibo,alexa,computer
0,Exclamation,4,2,4
1,Follow-Up Question,8,1,1
2,Relevancy,18,2,5
3,Judgemental Verbal,0,0,0
4,Glance at Others,26,19,11
5,Judgemental Eyebrow,0,0,1
6,Mhm,0,1,1
7,Insult,0,0,0
8,Waving,1,0,0
9,Imitation Dance,4,1,0


Export tables to be used in R:

In [106]:
for i in range(len(tableList)): 
    tableList[i].to_csv(tableNames[i] + "ContingencyTable.csv", index = False)